In [2]:
# Memory Allocation for using GPU 
import tensorflow as tf 
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model
from keras.layers import Input, concatenate, Dropout, Reshape, Permute, Activation, ZeroPadding2D, Cropping2D, Add
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, DepthwiseConv2D
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential # using Sequential
import numpy as np
import os


1 Physical GPUs, 1 Logical GPUs
Using TensorFlow backend.


## Load Dataset

In [3]:
DATA_PATH = os.path.abspath("..\\..\\00_MLDL\\00_BreadBrother\\05_Human_Segmentation")

IMG_H = 256
IMG_W = 256
BATCH_SIZE = 16

x_train = np.load(os.path.join(DATA_PATH, "preprocessed_dataset\\x_train.npz"))['data'].astype(np.float32) # 12706, 256, 256, 3
y_train = np.load(os.path.join(DATA_PATH, "preprocessed_dataset\\y_train.npz"))['data'].astype(np.float32) # 12706, 256, 256, 2

x_val = np.load(os.path.join(DATA_PATH, "preprocessed_dataset\\x_val.npz"))['data'].astype(np.float32) # 5000, 256, 256, 3
y_val = np.load(os.path.join(DATA_PATH, "preprocessed_dataset\\y_val.npz"))['data'].astype(np.float32) # 5000, 256, 256, 2 : why 2-ch? using softmax, 1st ch : human 0, 2nd ch :human 1
# for customize, using 2nd channel

print(f"Training Data Shape: {x_train.shape}")
print(f"Validation Data Shape: {x_val.shape}")

Training Data Shape: (12706, 256, 256, 3)
Validation Data Shape: (5000, 256, 256, 3)


In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255., # normalize 0~1
    brightness_range=[0.7, 1.3] # Data Augmentation
)

val_datagen = ImageDataGenerator(
    rescale=1./255.
)

train_gen = train_datagen.flow(
    x_train,
    y_train,
    batch_size = BATCH_SIZE,
    shuffle = True
)

val_gen = val_datagen.flow(
    x_val,
    y_val,
    batch_size = BATCH_SIZE,
    shuffle = False
)

## Build Model

In [15]:
inputs = Input(shape=(IMG_H, IMG_W, 3))

# Encode
conv1_1 = Conv2D(32, kernel_size=3, activation="relu", padding="same", kernel_initializer="orthogonal")(inputs) # Output shape (256,256,32)
# conv1_1 = Dorpout(0.2)(conv1_1)
conv1_2 = Conv2D(64, kernel_size=3, activation="relu", padding="same", kernel_initializer="orthogonal")(conv1_1) # Output shape (256,256,64)
pool1 = MaxPooling2D(pool_size=2)(conv1_2) # Output shape (128,128,64)

shortcut_1 = pool1

conv2_1 = Conv2D(64, 3, activation="relu", padding="same", kernel_initializer="orthogonal")(pool1) # Output shape (128, 128, 64)
# conv2_1 = Dorpout(0.2)(conv2_!)
conv2_2 = Conv2D(128, 3, activation="relu", padding="same", kernel_initializer="orthogonal")(conv2_1)
# Output shape (128, 128, 128)
pool2 = MaxPooling2D(pool_size=2)(conv2_2) # Output shape (64, 64, 128)

shortcut_2 = pool2

conv3_1 = Conv2D(128, 3, activation="relu", padding="same", kernel_initializer="orthogonal")(pool2) # Output shape (64, 64, 128)
# conv3_1 = Dropout(0.2)(conv3_1)
conv3_2 = Conv2D(256, 3, activation="relu", padding="same", kernel_initializer="orthogonal")(conv3_1)# Output shape (64, 64, 256)
pool3 = MaxPooling2D(pool_size=2)(conv3_2) # Output shape(32, 32, 256)

shortcut_3 = pool3

mid_1 = DepthwiseConv2D(3, activation="relu", padding="same", kernel_initializer="orthogonal")(pool3) # Output shape (32, 32, 256)
mid_2 = Conv2D(256, 1, activation="relu", padding="same", kernel_initializer="orthogonal")(mid_1) # Output shape (32, 32, 256)
mid_3 = DepthwiseConv2D(3, activation="relu", padding="same", kernel_initializer="orthogonal")(mid_2) # Output shape (32, 32, 256)
mid_4 = Conv2D(256, 1, activation="relu", padding="same", kernel_initializer="orthogonal")(mid_3) # Output shape (32, 32, 256)

mid_5 = Add()([shortcut_3, mid_4]) # (32,32,256) + (32,32,256) = (32,32,256)

# Decode
up8_1 = UpSampling2D(size=2)(mid_5) # Output shape (64, 64, 256)
up8_2 = concatenate([up8_1, conv3_2], axis=-1) # (64,64,256) + (64,64,256) = (64, 64, 512)



